In [3]:
import sys
sys.path.insert(1, '..')
from cocoa.geo import GeoManager as gm

In [4]:
help(gm)

Help on class GeoManager in module cocoa.geo:

class GeoManager(builtins.object)
 |  GeoManager(standard='iso2')
 |  
 |  GeoManager class definition. No inheritance from any other class.
 |  
 |  It should raise only CocoaError and derived exceptions in case
 |  of errors (see cocoa.error)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, standard='iso2')
 |      __init__ member function, with default definition of
 |      the used standard. To get the current default standard,
 |      see get_list_standard()[0].
 |  
 |  first_db_translation(self, w, db)
 |      This function helps to translate from country name to
 |      standard for specific databases. It's the first step
 |      before final translation.
 |      
 |      One can easily add some database support adding some new rules
 |      for specific databases
 |  
 |  get_list_db(self)
 |      return supported list of database name for translation of
 |      country names to standard.
 |  
 |  get_list_output(self)
 |   

In [7]:
g=gm()

In [8]:
g.get_list_db()

[None, 'jhu', 'worldometers', 'owid']

In [9]:
g.get_list_output()

['list', 'dict', 'pandas']

In [10]:
g.get_list_standard()

['iso2', 'iso3', 'name', 'num']

In [11]:
g.get_standard()

'iso2'

In [12]:
g.set_standard('name')

'name'

In [13]:
g.set_standard('oups') # the standard is not in the standard list 

CocoaKeyError: 'GeoManager.set_standard error, "oups not managed. Please see get_list_standard() function'

In [14]:
g.get_standard()

'name'

In [15]:
g.to_standard('england')

['United Kingdom']

In [16]:
g.to_standard('European Union') # the name does not exist as a country

CocoaLookupError: No country match the key "European Union". Error.

In [20]:
g.to_standard('European Union',interpret_region=True) # interpretation of region, and output as dict

['Austria',
 'Belgium',
 'Bulgaria',
 'Cyprus',
 'Czechia',
 'Germany',
 'Denmark',
 'Spain',
 'Estonia',
 'Finland',
 'France',
 'Greece',
 'Croatia',
 'Hungary',
 'Ireland',
 'Italy',
 'Lithuania',
 'Luxembourg',
 'Latvia',
 'Malta',
 'Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Slovenia',
 'Sweden']

In [18]:
g.to_standard('french') # difficult interpretation, getting first item

../cocoa/geo.py:163: UserWarning: Caution. More than one country match the key "french" : ['France, ', 'French Guiana, ', 'French Polynesia, ', 'French Southern Territories, ', 'Saint Martin (French part), '], using first one.

  warnings.warn('Caution. More than one country match the key "'+\


['France']

In [26]:
g.to_standard(['spain'],output='pandas')

ValueError: arrays must all be same length